## Convolutional Neural Networks in PyTorch

**Contents**
1. Getting data using Torchvision
2. Creating a CNN in PyTorch
3. Training our Network

### Setup and Installations

In [ ]:
!pip install numpy pandas torch torchvision pillow

### Getting Data with Torchvision

In [1]:
# make a directory to store MNIST images
import os

os.makedirs("./data", exist_ok=True)


In [2]:
from torchvision.datasets import MNIST

train_data = MNIST("./data", train=True, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4845807.61it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 55388977.51it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 6054886.53it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 10415816.71it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [8]:
# check our data
from PIL import Image
import matplotlib.pyplot as plt

data_item = train_data[0]

img = data_item[0]
target = data_item[1]

display(img)
print(target)

5


In [9]:
from pandas import Series

targets = Series([item[1] for item in train_data])

targets.value_counts()

1    6742
7    6265
3    6131
2    5958
9    5949
0    5923
6    5918
8    5851
4    5842
5    5421
Name: count, dtype: int64

### Creating a CNN in PyTorch

We are creating the `LeNet` architecture, which was one of the first proposed convolution neural network architectures. It consists of the following blocks:

- 2 conv blocks: 5x5 Convolution -> ReLU -> 2x2 pooling
- A feedforward classifier: linear layer -> ReLU -> linear layer -> Softmax

**Note:** we will not use batch normalization here, but you can add it after each pooling layer using `nn.BatchNorm2d` in PyTorch

**Overview of Softmax:**

We use softmax when predicting probabilities of different classifiers. The softmax function takes a vector as input and normalizes it such that all values in the vector sum to 1 (which is useful for classification probabilities)

In [10]:
# imports

import torch
from torch import nn, Tensor
import torch.nn.functional as F

In [11]:
# testing softmax
v = torch.rand(5) * 10
v

tensor([2.5512, 7.6166, 3.1391, 1.1121, 4.6326])

In [14]:
# softmax

v_sf = F.softmax(v, dim=-1) # on the last dimension, required for softmax due to updates
v_sf

tensor([0.0059, 0.9348, 0.0106, 0.0014, 0.0473])

In [15]:
torch.sum(v_sf)

tensor(1.)

**Building our Network in PyTorch**

To build our network in PyTorch, we're going to need a few building blocks:

- `nn.Conv2d(in_channels, out_channels, kernel_size, padding, stride)` - the base convolution operator. The in and out channels determines how deep the model is. We start with three channels (3), but we usually scale to 64/128 channels. LeNet uses a kernel size of 5x5, which requires a padding of 2 to keep the output the same size
- `nn.MaxPool2d(kernel_size, stride)` - the base pooling architecture. Max pooling has shown the best performance of all algorithms. We go across in 2x2 tiles and skip intermediate sections to get information from all sections individually
- `nn.ReLU()` - the relu activation function. We can use this as a module or the function version (`nn.functional.relu()`)
- `nn.Linear(in_dimensions, out_dimensions)` - a linear layer that we will use at the end of the convolution feature extractors
- `nn.Softmax(dimension)` - the softmax function, but as a module instead of the functional version (`nn.functional.softmax()`)

In [16]:
# creating our PyTorch network

class LeNet(nn.Module):
    def __init__(self, n_channels: int, n_classes: int) -> None:
        super().__init__()
        
        # create our first set of convolution layers
        self.conv1 = nn.Conv2d(in_channels=n_channels, out_channels=20, kernel_size=(5, 5)) # 5x5 kernel that takes the input channels and goes to 20 channels
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # the stride allows us to skip intermediates, so we select every unique 2x2 grid
        
        # create our second set of convolution layers
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5)) # create more channels for more features to extract
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # create our linear layers for classification
        self.linear1 = nn.Linear(in_features=800, out_features=500) # the number of in_features is calculated based on the flattened output of all images across all 50 channels after processing
        self.linear2 = nn.Linear(in_features=500, out_features=n_classes) # the class predictor
        self.softmax = nn.Softmax(dim=-1) # -1 selects the last dimension
        
    def forward(self, image: Tensor) -> Tensor:
        # run the first convolution
        out = self.conv1(image)
        out = F.relu(out)
        out = self.pool1(out)
        
        # run the second convolution
        out = self.conv2(out)
        out = F.relu(out)
        out = self.pool1(out)
        
        # run the feedforward network
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        
        return self.softmax(out)

### Training our Network